In [10]:
BEARER_TOKEN = 'AAAAAAAAAAAAAAAAAAAAAOacaQEAAAAAfief77Bv8V%2BLRCusIsMqJDdquEI%3DKEg3xZEAr2ClMqxR2RHTJRq2kn7PbkOJR5UmIu68rdA16X3hwN'

In [11]:
import tweepy
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import warnings
import re
import nltk
from nltk.corpus import stopwords
import nltk as nlp
from nltk.stem.porter import PorterStemmer
import neattext.functions as nfx
from tkinter import *
from tkinter import ttk
import tkinter.messagebox
from tkinter.scrolledtext import ScrolledText
import pyttsx3 
import time
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import classification_report, confusion_matrix
warnings.filterwarnings("ignore")

In [12]:
#initilization
client = tweepy.Client(bearer_token=BEARER_TOKEN)

lemma=nlp.WordNetLemmatizer()
stemmer = PorterStemmer()
def data_cleaner(tweet):
    #     remove urls
    tweet=re.sub(r'http\S+', ' ',tweet)
    
    #     remove html tags
    tweet=re.sub(r'<.*?>',' ',tweet)
    
    #     remove digits
    tweet=re.sub(r'\d+',' ', tweet)
    
    #     remove hastags
    tweet=re.sub(r'#\w+',' ',tweet)
    
    #     remove mentions
    tweet=re.sub(r'@\w+', ' ',tweet)
    
    #     remove punctions
    tweet=re.sub('[^a-zA-Z]', ' ', tweet)
    
    tweet=tweet.lower()
    tweet=nltk.word_tokenize(tweet,'english')
    tweet=[word for word in tweet if not word in set(stopwords.words('english'))]
    #tweet=[stemmer.stem(word) for word in tweet]
    tweet=[lemma.lemmatize(word) for word in tweet]
    tweet=' '.join(tweet)
    return tweet

In [13]:
train_df=pd.read_csv("Corona_NLP_train.csv",encoding="latin1")[['Location','OriginalTweet','Sentiment']].head(10000)
clean_train_df = train_df['OriginalTweet'].apply(data_cleaner)
train_df['clean_tweet'] = clean_train_df
# drop if clean_tweet is empty
train_df= train_df[train_df['clean_tweet'] != 0]
X=train_df['clean_tweet'].copy()
y=train_df['Sentiment'].copy()

train_majority = train_df[y==-1]
train_minority = train_df[y==1]
train_medium   = train_df[y==0]
train_minority_upsampled = resample(train_minority,
                                 replace=True,
                                 n_samples=len(train_majority),
                                 random_state=123)
train_upsampled = pd.concat([train_minority_upsampled, train_majority,train_medium])
label = train_upsampled['label']
tweet = train_upsampled['clean_tweet']
pipeline_sgd = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf',  TfidfTransformer()),
    ('nb', SGDClassifier()),
])
X_train, X_test, y_train, y_test = train_test_split(tweet,label,test_size = 0.5,random_state = 0)
model = pipeline_sgd.fit(X_train, y_train)
y_predict = model.predict(X_test)
mat = confusion_matrix(y_test,y_predict)


KeyError: 'label'

In [ ]:
import tkinter
from typing import Callable
from matplotlib.backend_bases import key_press_handler
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure

def trainheatmap():
    trainheatwin = Toplevel(master)
    trainheatwin.configure(background='#CDCDCD')
    def init_gui(root, update_function: Callable) -> FigureCanvasTkAgg:
        def event_key_press(event):
            print("you pressed {}".format(event.key))
            update_function()
            key_press_handler(event, canvas)

        # create empty figure and draw
        init_figure = create_figure()
        canvas = FigureCanvasTkAgg(init_figure, master=root)
        canvas.draw()
        canvas.get_tk_widget().pack(side=tkinter.TOP, fill=tkinter.BOTH, expand=1)
        # call key press event
        canvas.mpl_connect("key_press_event", event_key_press)
        return canvas


    def create_figure() -> Figure:
        # generate some data
        matrix = mat
        # plot the data
        figure = Figure(figsize=(6, 6))
        ax = figure.subplots()
        categories = ['Negative','Neutral','Positive']
        sns.heatmap(matrix.T, square=True, annot=True, fmt='d', cbar=False, 
        xticklabels=categories, yticklabels=categories,ax=ax)
        return figure


    def redraw_figure():
        figure = create_figure()
        canvas.figure = figure
        canvas.draw()


    sns.set()
    canvas = init_gui(trainheatwin, update_function=redraw_figure)
    T = Text(trainheatwin, height = 10, width = 82)
    T.pack()
    T.insert(END, classification_report(y_test,y_predict))

                

In [ ]:
import tkinter as tk
import matplotlib


matplotlib.use('TkAgg')

from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import (
    FigureCanvasTkAgg,
    NavigationToolbar2Tk
)
def clean():
    global df
    clean_df = df['tweets'].apply(data_cleaner)
    df['clean_tweet'] = clean_df 
    RER= model.predict(df.clean_tweet)
    df['Analysis'] = RER
    
def downloandTweets():
    global df
    keyword=e1.get()
    query = ' -is:retweet lang:en -has:images -has:videos'
    keyword += query
    query = keyword
    try:
        response = client.search_recent_tweets(query=query , max_results = 100, tweet_fields = ['lang'])
        tweets = pd.DataFrame([tweet.text for tweet in response.data],columns=['tweets'])
        df = tweets
        return df     
    except:
        tkinter.messagebox.showinfo("Warning",  "There is no tweets with relative your keywords")
      
         
def showDownloandedTweets(df):
    # Create an instance of tkinter frame
    newWindow = Toplevel(master)
    newWindow.configure(background='#CDCDCD')

    # Set the geometry
    newWindow.title("Downloaded Tweets")
    newWindow.geometry('1080x340')

    # Add a text widget
    text=ScrolledText(newWindow,wrap = WORD,width = 80,height = 10,background='#CDCDCD', font=('arial',15,'bold'))

    # Add some text in the text widget
    
    for i in df.tweets:
        text.insert(END, i+"\n_______________________________________________________________________________\n")

    text.pack()
    def close_section1():
        newWindow.destroy()
    button1=Button(newWindow, text="Analyze the tweets",background='#CDCDCD', font=('arial',15,'bold'),fg="black",command=lambda:[close_section1(), analyzeDTweets()])
    button1.config( height = 3, width = 25 )
    button1.pack(side=BOTTOM)
    
def analyzeDTweets():
    clean()
    analyzeWindow = Toplevel(master)
    analyzeWindow.title("Analysing Tweets")
    analyzeWindow.geometry('1080x1080')
    analyzeWindow.configure(background='#CDCDCD')
    pos = 0
    neg = 0
    neu = 0
    for i in df["Analysis"]:
        if i < 0:
            neg+=1
        elif i == 0:
            neu+=1
        else:
            pos+=1
    data = {
            'Negative': neg,
            'Neutral': neu,
            'Positive': pos,
        }
    languages = data.keys()
    popularity = data.values()

        # create a figure
    figure = Figure(figsize=(6, 4), dpi=100)

        # create FigureCanvasTkAgg object
    figure_canvas = FigureCanvasTkAgg(figure, analyzeWindow)

        # create the toolbar
    NavigationToolbar2Tk(figure_canvas, analyzeWindow)

        # create axes
    axes = figure.add_subplot()

        # create the barchart
    axes.bar(languages, popularity)
    axes.set_title('3 Label in Downloaded Tweets')
    axes.set_ylabel('Sentiment')


    figure_canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=1)
    button1=Button(analyzeWindow, text="Word in Specific Label",background='#CDCDCD',
                   font=('arial',15,'bold'),fg="black",
                   command=wordInLabel)
    button1.config( height = 3, width = 25 )
    button1.pack()

def wordInLabel():
    wordLabel = Toplevel(master)
    wordLabel.title("Detail Tweets")
    wordLabel.geometry('648x340')
    wordLabel.configure(background='#CDCDCD')
    button1=Button(wordLabel, text="Back of Negative Words",background='#CDCDCD', font=('arial',15,'bold'),fg="black",command=lambda:[chooce(-1)])
    button2=Button(wordLabel, text="Back of Neutral  Words",background='#CDCDCD', font=('arial',15,'bold'),fg="black",command=lambda:[chooce(0)])
    button3=Button(wordLabel, text="Back of Positive Words",background='#CDCDCD', font=('arial',15,'bold'),fg="black",command=lambda:[chooce(1)])
    button11=Button(wordLabel, text=" Negative Tweets",background='#CDCDCD', font=('arial',15,'bold'),fg="black",command=lambda:[chooce(11)])
    button22=Button(wordLabel, text="  Neutral Tweets",background='#CDCDCD', font=('arial',15,'bold'),fg="black",command=lambda:[chooce(22)])
    button33=Button(wordLabel, text=" Positive Tweets",background='#CDCDCD', font=('arial',15,'bold'),fg="black",command=lambda:[chooce(33)])
    button4=Button(wordLabel, text=" Top 10 Hashtag",background='#CDCDCD', font=('arial',15,'bold'),fg="black",command=lambda:[chooce(2)])
    button1.grid(row=0,column=1)
    button2.grid(row=1,column=1)
    button3.grid(row=2,column=1)
    button11.grid(row=0,column=2)
    button22.grid(row=1,column=2)
    button33.grid(row=2,column=2)
    button4.grid(row=1,column=3)
    all_words = ""
    def chooce(n):
        if n < 0 :
            all_words = " ".join([sentence for sentence in df['clean_tweet'][df['Analysis']==-1]])
            wordcloud = WordCloud(width=800, height=500, random_state=42, max_font_size=100).generate(all_words)    
            plt.figure(figsize=(15,8))
            plt.imshow(wordcloud, interpolation='bilinear')
            plt.axis('off')
            plt.show()

        if n == 0:
            all_words = " ".join([sentence for sentence in df['clean_tweet'][df['Analysis']==0]])
            wordcloud = WordCloud(width=800, height=500, random_state=42, max_font_size=100).generate(all_words)    
            plt.figure(figsize=(15,8))
            plt.imshow(wordcloud, interpolation='bilinear')
            plt.axis('off')
            plt.show()
        if n == 1:
            all_words = " ".join([sentence for sentence in df['clean_tweet'][df['Analysis']==1]])
            wordcloud = WordCloud(width=800, height=500, random_state=42, max_font_size=100).generate(all_words)    
            plt.figure(figsize=(15,8))
            plt.imshow(wordcloud, interpolation='bilinear')
            plt.axis('off')
            plt.show()
        if n == 11:
            words = Toplevel(wordLabel)
            words.title("Negative Tweets")
            words.geometry('1080x340')
            label_sepetereter= Label(words,text="Negative Tweets",font=('Courier New',20),fg="black")
            text=ScrolledText(words,wrap = WORD,width = 80,height = 10,background='#CDCDCD', font=('arial',15,'bold'))
            for i in df['tweets'][df['Analysis']==-1]:
                text.insert(END, i+"\n_______________________________________________________________________________\n")
            label_sepetereter.pack()
            text.pack()
        if n == 22:
            words = Toplevel(wordLabel)
            words.title("Neutral Tweets")
            words.geometry('1080x340')
            label_sepetereter= Label(words,text="Neutral Tweets",font=('Courier New',20),fg="black")
            text=ScrolledText(words,wrap = WORD,width = 80,height = 10,background='#CDCDCD', font=('arial',15,'bold'))
            for i in df['tweets'][df['Analysis']==0]:
                text.insert(END, i+"\n_______________________________________________________________________________\n")
            label_sepetereter.pack()
            text.pack()
        if n == 33:
            words = Toplevel(wordLabel)
            words.title("Positive Tweets")
            words.geometry('1080x340')
            label_sepetereter= Label(words,text="Positive Tweets",font=('Courier New',20),fg="black")
            text=ScrolledText(words,wrap = WORD,width = 80,height = 10,background='#CDCDCD', font=('arial',15,'bold'))
            for i in df['tweets'][df['Analysis']==1]:
                text.insert(END, i+"\n_______________________________________________________________________________\n")
            label_sepetereter.pack()
            text.pack()
        if n == 2:
            extract_hashtags =df['tweets'].apply(nfx.extract_hashtags)
            extract_hashtags = sum(extract_hashtags, [])
            freq = nltk.FreqDist(extract_hashtags)
            d = pd.DataFrame({'Hashtag': list(freq.keys()),'Count': list(freq.values())})
            d = d.nlargest(columns='Count', n=10)
            plt.figure(figsize=(12,6))
            plt.title('Top 10 Hashtags',fontsize=15)
            plt.xticks(rotation=45)
            sns.barplot(data=d, x='Hashtag', y='Count')
            plt.show()
            


In [ ]:
def reset_button():
    label_2.config(text = "Press for Downloanding")
    btn1.configure(text = "Download" ,command=lambda:[downloandTweets, show_button()])
    
def show_button():
    label_2.config(text = " Press for Showing")
    if len(e1.get()) != 0:
        df = downloandTweets()
        btn1.configure(text = "show", command=lambda:[showDownloandedTweets(df), reset_button()])
    else:
        tkinter.messagebox.showinfo("Warning",  "Please enter keyword")
#MAIN
master = Tk() 
master.title("Sentiment Analysis On Twitter")
master.geometry('750x200')
master.configure(background='#CDCDCD')

label_1 = Label(master, text='Keyword:',background='#CDCDCD', font=('arial',15,'bold')).grid(row=0,column=0) 
e1 = Entry(master,bd =10,width=30,font=20) 
e1.grid(row=0, column=5) 
label_2 = Label(master,text="Press for Downloanding",background='#CDCDCD', font=('arial',15,'bold'))
label_3 = Label(master,text="Click for showing trained data",background='#CDCDCD', font=('arial',15,'bold'))
label_sepetereter= Label(master,text="_____________________",font=('Courier New',20),bg="blue",fg="blue")
    

btn1=Button(master,text=" Download ",bg="blue",background='#CDCDCD', font=('arial',15,'bold'),fg="black",command=lambda:[downloandTweets, show_button()])#command=lambda:[function1(), function2()]
btn2=Button(master,text="Click",bg="blue",background='#CDCDCD', font=('arial',15,'bold'),fg="black",command=trainheatmap)
label_2.grid(row=2,column=0)
label_3.grid(row=4,column=0)
label_sepetereter.grid(row=3,column=0)
btn1.grid(row=2,column=5)
btn2.grid(row=4,column=5)


mainloop()

In [ ]:
df